
# Sentiment - Analyser #


## Loading Data ##

Before starting with the machine learning and sentiment analysis
we first have to load the data and clean it! 

In [1]:

from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'

driver = GraphDatabase.driver(uri, auth=(user, password))

# resetting database

words = []
comments = []
scores = []

with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    result = _q("MATCH (n:Author)-[c:COMMENTED]->(v:Video) RETURN n, c, v") # remove all graphs and nodes! BE CAREFUL!
 
    for record in result:
        newComment = (("%s | %s" % (record['v']['title'],record['c']['text'])).lower()
            .replace('❤️', ' heart ').replace('💯', ' 100 ').replace('❤', ' heart ')
            .replace('🙏', ' pray ').replace('😘', ' kiss ').replace('🤗', ' happy ')
            .replace('💥', ' boom ').replace('✔️', ' like ').replace('😍', ' love ')
            .replace('🐱', ' cat ').replace('💔', ' broken heart ').replace('😵', ' confused ') 
            .replace('😄', ' awesome ').replace('👍', ' thumbs up ').replace('😎', ' cool ')
            .replace('🐷', ' pig ').replace('🤘', " rock'n roll ").replace('🤣', ' laughing hard ')
            .replace('😩', ' oh no ').replace('💎', ' diamond ').replace('😊', ' nice ')
            .replace('☺️', ' very nice ').replace('🙃', ' upside down smile ').replace('🤔', ' not sure ')
            .replace('😂', ' laughing ')
            .replace('!!!!', '!').replace('!!!', '!').replace('!!', '!')
            .replace('????', '?').replace('???', '?').replace('??', '?')
            .replace('oooo','o').replace('oooo','o').replace('ooo','o').replace('oo','o')
            .replace('..','...').replace('......','...').replace('....','...').replace('....','...').replace('...',' ... ')
            .replace('    ', ' ').replace('   ', ' ').replace('  ', ' ')
            .split() 
        )
        words.extend(newComment)
        comments.append(newComment)
        scores.append(float(record['c']['score']))
    
    #---------------------------------------
driver.close()

#print(comments)
#print(scores)

## Let's clean the text with spark ##

The comments are currently still full of redundancies and
many uncommon tokens / word.
In order to be able to filter the comments we use spark to count their occurrences.

In [2]:
from pyspark import SparkContext
 
sc = SparkContext("local", "First App")
 
wordsRDD = sc.parallelize(words, 6)

zipped = (wordsRDD
                       .map(lambda word : (word,1))
                       .reduceByKey( lambda a, b : a+b )
                       .collect())
sc.stop()                  

occurrences = {}

for word, count in zipped:
    occurrences[word] = count


...and now let's remove rare tokens / words in order to cap the maximum amount of possible words a bit...

In [3]:

def less_than_three(word):
    return occurrences[word] > 3

for i, comment in enumerate(comments): 
    comments[i] = list(filter(less_than_three, comment))

In [4]:
print(comments)

[['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'great', 'series'], ['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'is'], ['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'could', 'we', 'have', 'the', 'code'], ['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'please', '2', '3', 'videos'], ['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'your', 'is', 'to', 'more', 'videos', 'on', 'python'], ['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'is'], ['sentiment', 'analysis', 'python', '-', '1', '-', 'introduction', 'to', 'emotion', 'analysis', '(nlp)', '|', 'can', 'please', 'give', 'me', 'code', 'for', 'different', 'emotions',

---
# Sentiment Analysis with Pytorch #

Let's train a neural network on our data!

In [5]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [6]:
class LSTMScorePredictor(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_dim, do_softmax=True):
        super(LSTMScorePredictor, self).__init__()
        self.do_softmax = do_softmax
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.lin = nn.Linear(hidden_dim, output_dim)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        out = self.lin(lstm_out.view(len(sentence), -1))
        if self.do_softmax: out = F.log_softmax(out, dim=1)
            
        return out

## Working with the model: ##

In [7]:

# Word to index encoding...
class Encoder:
    
    def __init__(self, training_data):
        self.word_to_ix = {}
        # For each words-list (sentence) and tags-list in each tuple of training_data
        for sent, label in training_data:
            for word in sent:
                if word not in self.word_to_ix:  # word has not been assigned an index yet
                    self.word_to_ix[word] = len(self.word_to_ix)  # Assign each word with a unique index
        print(self.word_to_ix)
        self.label_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

    def sequence_words(self, seq):
        idxs = [self.word_to_ix[w] for w in seq]
        return torch.tensor(idxs, dtype=torch.long)
    
    
    def sequence_labels(self, seq):
        idxs = [self.label_to_ix[w] for w in seq]
        return torch.tensor(idxs, dtype=torch.long)

## Unit-Testing the Model: ##

In [8]:
# Example training data:

training_data = [ 
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
enc = Encoder(training_data)

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

model = LSTMScorePredictor(
    embedding_dim=EMBEDDING_DIM, 
    hidden_dim=HIDDEN_DIM, 
    vocab_size=len(enc.word_to_ix), 
    output_dim=len(enc.label_to_ix) 
)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = enc.sequence_words(training_data[0][0])
    tag_scores = model(inputs)
    print(tag_scores)
    
assert str(tag_scores) == """tensor([[-0.8818, -1.0841, -1.3953],
        [-0.9756, -0.9889, -1.3820],
        [-0.9249, -1.0800, -1.3325],
        [-0.8884, -1.1795, -1.2684],
        [-0.8897, -1.1504, -1.2993]])"""

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = enc.sequence_words(sentence)
        targets = enc.sequence_labels(tags)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = enc.sequence_words(training_data[0][0])
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(inputs)
    print(tag_scores)
    
assert str(inputs) == """tensor([0, 1, 2, 3, 4])"""
assert str(tag_scores) == """tensor([[-0.0259, -4.5027, -4.2358],
        [-4.5316, -0.0617, -3.0154],
        [-2.6551, -2.9622, -0.1301],
        [-0.1583, -3.9917, -2.0562],
        [-4.2915, -0.0241, -4.5954]])"""

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}
tensor([[-0.8818, -1.0841, -1.3953],
        [-0.9756, -0.9889, -1.3820],
        [-0.9249, -1.0800, -1.3325],
        [-0.8884, -1.1795, -1.2684],
        [-0.8897, -1.1504, -1.2993]])
tensor([0, 1, 2, 3, 4])
tensor([[-0.0259, -4.5027, -4.2358],
        [-4.5316, -0.0617, -3.0154],
        [-2.6551, -2.9622, -0.1301],
        [-0.1583, -3.9917, -2.0562],
        [-4.2915, -0.0241, -4.5954]])


## Doing some classical regression now! ##

When evaluating the sentiment we want the result to be somethi

In [9]:
# Example training data:

training_data = [ 
    ("The dog ate the apple".split(), torch.tensor([[-2], [3], [4], [-2], [6]])),
    ("Everybody read that book".split(), torch.tensor([[[-5], [-2], [0], [-11]]]))
]
enc = Encoder(training_data)

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

model = LSTMScorePredictor(
    embedding_dim=EMBEDDING_DIM, 
    hidden_dim=HIDDEN_DIM, 
    vocab_size=len(enc.word_to_ix), 
    output_dim=1,
    do_softmax=False
)
loss_function = nn.L1Loss()# instead of nn.NLLLoss() (which is for softmax output...)
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = enc.sequence_words(training_data[0][0])
    tag_scores = model(inputs)
    print(tag_scores)
    
 
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = enc.sequence_words(sentence)
        targets = tags# enc.sequence_labels(tags)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step() 
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = enc.sequence_words(training_data[0][0])
    pred1 = model(inputs)
 
    print(inputs)
    print(pred1)
    
    inputs = enc.sequence_words(training_data[1][0])
    pred2 = model(inputs)
 
    print(inputs)
    print(pred2)
     
assert pred1 == """tensor([[-1.9960],
        [ 3.0707],
        [ 4.1012],
        [-1.9682],
        [ 6.1665]])"""

assert pred2 == """tensor([[ -5.1595],
        [ -2.2969],
        [ -0.2990],
        [-11.1489]])"""

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}
tensor([[-0.1821],
        [-0.1732],
        [-0.2096],
        [-0.2086],
        [-0.1786]])


/home/lukashinterleitner/anaconda3/envs/neo4j/lib/python3.8/site-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 4, 1])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


tensor([0, 1, 2, 3, 4])
tensor([[-1.9960],
        [ 3.0706],
        [ 4.1011],
        [-1.9683],
        [ 6.1664]])
tensor([5, 6, 7, 8])
tensor([[ -5.1595],
        [ -2.2969],
        [ -0.2990],
        [-11.1489]])


AssertionError: 